Import the libraries needed

In [26]:
import pandas as pd
#from pandas.compat import StringIO
from pandas.tseries.offsets import DateOffset

from datetime import datetime
#ateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
#dateparse = lambda x: datetime.strptime(x '%Y-%m-%d')

import psutil




import numpy as np
import os
import pyodbc
pyodbc.drivers()

import csv

from sklearn.preprocessing import StandardScaler
import seaborn as sns

pd.options.display.max_rows = 1000

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 


### Database connections and SQL to retrieve the various data needed.

In [27]:
pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
os.path.abspath(os.getcwd())

2024-08-05 21:40:05.420354
svmem(total=16756752384, available=5083897856, percent=69.7, used=11672854528, free=5083897856)


'C:\\Users\\m033\\OneDrive - GWLE\\Documents\\GitHub\\Data-Science-Thesis'

# 1. Read back the population with switches data.

In [28]:
# Read back the comma separated values file.

# giving directory name
filename = input_path + '\Full_Population_With_Switch_Flags.csv'

#import CSV file and specify dtype of each column
# https://www.statology.org/pandas-read-csv-dtype/
df_Horizon = pd.read_csv(filename,dtype = {'ContractNumber': str,'ProdCat': str,'ProdCode': str,'PolicyStatus': str,'Switch_Flag': int})
# Set all dates to proper dates
df_Horizon['SnapshotDt'] = pd.to_datetime(df_Horizon['SnapshotDt'])
df_Horizon['CoverEndDt'] = pd.to_datetime(df_Horizon['CoverEndDt'])

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_Horizon.dtypes)
#df_Horizon.head(5)

2024-08-05 21:40:24.033063
svmem(total=16756752384, available=4345774080, percent=74.1, used=12410978304, free=4345774080)
ContractNumber            object
ProdCat                   object
ProdCode                  object
PolicyStatus              object
CoverEndDt        datetime64[ns]
SnapshotDt        datetime64[ns]
Switch_Flag                int32
dtype: object


# 2. Get supplementary policy features. 

  -  Add policy level features.
  -  Tax
  -  Asset related features
  -  Indexation and premium.
  -  Broker and Commission 
  -  Client data
  -  customer Market segmentation


## 2.1 Tax, Fee and Risk features.
### All tax and Fee features wre analysed and only one Risk feature remains.

In [29]:
# Read back the comma separated values file.

# giving directory name
filename = input_path + '\Tax_Fee_Risk_data.csv'

#df_Tax_Fee_Risk_data = pd.read_csv(filename, delimiter=',',low_memory=False, header=0)
df_Tax_Fee_Risk_data = pd.read_csv(filename,dtype = {'ContractNumber': str,'Policy_Fee_Type_Desc': str})
# Set all dates to proper dates


print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_Tax_Fee_Risk_data.dtypes)

# No displays to protect PII data
#df_Tax_Fee_Risk_data.head(5)

2024-08-05 21:40:24.237069
svmem(total=16756752384, available=4325740544, percent=74.2, used=12431011840, free=4325740544)
ContractNumber          object
Policy_Fee_Type_Desc    object
dtype: object


## Merge in the Tax, Fee and risk data.

In [30]:
df_Horizon_All_Features = pd.merge(df_Horizon, df_Tax_Fee_Risk_data, left_on = ['ContractNumber'], right_on = ['ContractNumber'] , how = 'left')
del df_Tax_Fee_Risk_data
del df_Horizon

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

%who_ls DataFrame 

# No displays to protect PII data
#df_Horizon_All_Features.head(5)

2024-08-05 21:40:30.242629
svmem(total=16756752384, available=4172902400, percent=75.1, used=12583849984, free=4172902400)


['df_Horizon_All_Features']

## 2.2 Asset features.
### All asset features were analysed and only one feature remains.

In [31]:
# Read back the comma separated values file.

# giving directory name
filename = input_path + '\Asset_data.csv'

df_Asset = pd.read_csv(filename,dtype = {'ContractNumber': str,'PricingVariant': int})
# Set all dates to proper dates

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_Asset.dtypes)

# No displays to protect PII data
#df_Asset.head(5)

2024-08-05 21:40:30.419815
svmem(total=16756752384, available=4178350080, percent=75.1, used=12578402304, free=4178350080)
ContractNumber    object
PricingVariant     int32
dtype: object


## Merge in the asset data.

In [32]:
df_Horizon_All_Features = pd.merge(df_Horizon_All_Features, df_Asset, left_on = ['ContractNumber'], right_on = ['ContractNumber'] , how = 'left')

df_Horizon_All_Features = df_Horizon_All_Features.rename(columns={'Policy_Fee_Type_Desc': 'PolicyFeeType'})

del df_Asset

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
print(df_Horizon_All_Features.dtypes)

%who_ls DataFrame 

# No displays to protect PII data
#df_Horizon_All_Features.head(5)

2024-08-05 21:40:37.059749
svmem(total=16756752384, available=3670003712, percent=78.1, used=13086748672, free=3670003712)
ContractNumber            object
ProdCat                   object
ProdCode                  object
PolicyStatus              object
CoverEndDt        datetime64[ns]
SnapshotDt        datetime64[ns]
Switch_Flag                int32
PolicyFeeType             object
PricingVariant           float64
dtype: object


['df_Horizon_All_Features']

## 2.3 Indexation and premium features.
### All Indexation and premium features were analysed and??? only one feature remains.

In [33]:
# Read back the comma separated values file.

# giving directory name
filename = input_path + '\Premium_data.csv'

df_premium = pd.read_csv(filename,dtype = {'ContractNumber': str,
                                           'PayFreq': int},low_memory=False, header=0)
# Set all dates to proper dates
df_premium['AnniversaryDt'] = pd.to_datetime(df_premium['AnniversaryDt'])
df_premium['IndexationStartDt'] = pd.to_datetime(df_premium['IndexationStartDt'])
df_premium['PremHolStartDt'] = pd.to_datetime(df_premium['PremHolStartDt'])
df_premium['PremHolEndDt'] = pd.to_datetime(df_premium['PremHolEndDt'])


print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_premium.dtypes)

# No displays to protect PII data
#df_premium.head(5)

2024-08-05 21:40:38.029437
svmem(total=16756752384, available=3672526848, percent=78.1, used=13084225536, free=3672526848)
Contract_Number              object
PayFreq                       int32
AnniversaryDt        datetime64[ns]
IndexationPercent             int64
IndexationStartDt    datetime64[ns]
PremHolStartDt       datetime64[ns]
PremHolEndDt         datetime64[ns]
Premium                     float64
PremiumCalcMethod            object
dtype: object


## Merge in the Premium and Indexation data.

In [34]:
df_premium = df_premium.rename(columns={'Contract_Number': 'ContractNumber'})

df_Horizon_All_Features = pd.merge(df_Horizon_All_Features, df_premium, left_on = ['ContractNumber'], right_on = ['ContractNumber'] , how = 'left')

del df_premium

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
print(df_Horizon_All_Features.dtypes)

%who_ls DataFrame 


# No displays to protect PII data
#df_Horizon_All_Features.head(5)

2024-08-05 21:41:04.878613
svmem(total=16756752384, available=3373772800, percent=79.9, used=13382979584, free=3373772800)
ContractNumber               object
ProdCat                      object
ProdCode                     object
PolicyStatus                 object
CoverEndDt           datetime64[ns]
SnapshotDt           datetime64[ns]
Switch_Flag                   int32
PolicyFeeType                object
PricingVariant              float64
PayFreq                       int32
AnniversaryDt        datetime64[ns]
IndexationPercent             int64
IndexationStartDt    datetime64[ns]
PremHolStartDt       datetime64[ns]
PremHolEndDt         datetime64[ns]
Premium                     float64
PremiumCalcMethod            object
dtype: object


['df_Horizon_All_Features']

## 2.4  Broker and Commission.
### All features were analysed and all remains.

In [35]:
# Read back the comma separated values file.

# giving directory name
filename = input_path + '\Broker_and_Commission_data.csv'

df_Broker_and_Comm = pd.read_csv(filename,dtype = {'ContractNumber': str,
                                                   'ServicingBroker': str,
                                                   'InitialBroker': str,
                                                   'BrokerCategory': str},low_memory=False, header=0)

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_Broker_and_Comm.dtypes)

# No displays to protect PII data
#df_Broker_and_Comm.head(5)

2024-08-05 21:41:05.147700
svmem(total=16756752384, available=3366658048, percent=79.9, used=13390094336, free=3366658048)
ContractNumber      object
ServicingBroker     object
InitialBroker       object
BrokerCategory      object
InitialRate        float64
RenewalRate        float64
dtype: object


## Merge in the Broker and Commission data.

In [36]:
df_Horizon_All_Features = pd.merge(df_Horizon_All_Features, df_Broker_and_Comm, left_on = ['ContractNumber'], right_on = ['ContractNumber'] , how = 'left')

del df_Broker_and_Comm

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
print(df_Horizon_All_Features.dtypes)


# No displays to protect PII data
#df_Horizon_All_Features.head(5)

2024-08-05 21:41:16.032901
svmem(total=16756752384, available=3116773376, percent=81.4, used=13639979008, free=3116773376)
ContractNumber               object
ProdCat                      object
ProdCode                     object
PolicyStatus                 object
CoverEndDt           datetime64[ns]
SnapshotDt           datetime64[ns]
Switch_Flag                   int32
PolicyFeeType                object
PricingVariant              float64
PayFreq                       int32
AnniversaryDt        datetime64[ns]
IndexationPercent             int64
IndexationStartDt    datetime64[ns]
PremHolStartDt       datetime64[ns]
PremHolEndDt         datetime64[ns]
Premium                     float64
PremiumCalcMethod            object
ServicingBroker              object
InitialBroker                object
BrokerCategory               object
InitialRate                 float64
RenewalRate                 float64
dtype: object


In [37]:
%who_ls DataFrame 

['df_Horizon_All_Features']

## 2.5 Client data
### All features were analysed and most remains.

In [38]:
# Read back the comma separated values file.

# giving directory name
filename = input_path + '\Client_data.csv'


df_Client = pd.read_csv(filename,dtype = {'Policy_Number': str,
                                          'VIPType': str,
                                          'Nationality': str,
                                          'IsSmoker': bool,
                                          'PoliticallyExposed': str,
                                          'BirthPlace': str,
                                          'Gender': str,
                                          'MaritalStatus': str,
                                          'AddressCity': str,
                                          'PartnerType': str,
                                          'EUSanctioned': bool,
                                          'AddressDistrict': str,
                                          'Postal_Code_Name': str,
                                          'District': str,
                                          'State': str,
                                          'VN_MILIEU_CODE': str,
                                          'VN_TARGET_GROUP_CODE': str,
                                          'VN_MILIEU_DESCRIPTION': str,
                                          'VP_MILIEU_DESCRIPTION': str},low_memory=False, header=0)
# Set all dates to proper dates
df_Client['RiskCommDt'] = pd.to_datetime(df_Client['RiskCommDt'])
df_Client['DOB'] = pd.to_datetime(df_Client['DOB'])
df_Client['Postal_Code'] = df_Client['Postal_Code'].fillna(0).astype(int)

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_Client.dtypes)

# No displays to protect PII data
#df_Client.head(5)

2024-08-05 21:41:17.959782
svmem(total=16756752384, available=3089080320, percent=81.6, used=13667672064, free=3089080320)
Policy_Number                    object
RiskCommDt               datetime64[ns]
VIPType                          object
Nationality                      object
IsSmoker                           bool
PoliticallyExposed               object
BirthPlace                       object
Gender                           object
MaritalStatus                    object
PensionDisability                 int64
WOPDisability                     int64
AddressCity                      object
DOB                      datetime64[ns]
PartnerType                      object
EUSanctioned                       bool
AddressDistrict                  object
Postal_Code                       int32
Postal_Code_Name                 object
District                         object
State                            object
VN_MILIEU_CODE                   object
VN_TARGET_GROUP_CODE             obje

## Merge in the Client and address features as well as the Sinus Milieus and postcode features.

In [39]:
df_Client = df_Client.rename(columns={'Policy_Number': 'ContractNumber'})

df_Horizon_All_Features = pd.merge(df_Horizon_All_Features, df_Client, left_on = ['ContractNumber'], right_on = ['ContractNumber'] , how = 'left')

del df_Client

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
print(df_Horizon_All_Features.dtypes)


# No displays to protect PII data
#df_Horizon_All_Features.head(5)

2024-08-05 21:41:49.496052
svmem(total=16756752384, available=2774573056, percent=83.4, used=13982179328, free=2774573056)
ContractNumber                   object
ProdCat                          object
ProdCode                         object
PolicyStatus                     object
CoverEndDt               datetime64[ns]
SnapshotDt               datetime64[ns]
Switch_Flag                       int32
PolicyFeeType                    object
PricingVariant                  float64
PayFreq                           int32
AnniversaryDt            datetime64[ns]
IndexationPercent                 int64
IndexationStartDt        datetime64[ns]
PremHolStartDt           datetime64[ns]
PremHolEndDt             datetime64[ns]
Premium                         float64
PremiumCalcMethod                object
ServicingBroker                  object
InitialBroker                    object
BrokerCategory                   object
InitialRate                     float64
RenewalRate                     float

## Create the age and term columns.

In [40]:
# Set all dates to proper dates
#df_Horizon_All_Features['CoverEndDt'] = pd.to_datetime(df_Horizon_All_Features['CoverEndDt'])
#df_Horizon_All_Features['SnapshotDt'] = pd.to_datetime(df_Horizon_All_Features['SnapshotDt'])
#df_Horizon_All_Features['RiskCommDt'] = pd.to_datetime(df_Horizon_All_Features['RiskCommDt'])
#df_Horizon_All_Features['AnniversaryDt'] = pd.to_datetime(df_Horizon_All_Features['AnniversaryDt'])
#df_Horizon_All_Features['DOB'] = pd.to_datetime(df_Horizon_All_Features['DOB'])

# Calculate the term columns
df_Horizon_All_Features['Term'] = ((df_Horizon_All_Features.CoverEndDt - df_Horizon_All_Features.RiskCommDt)/np.timedelta64(1, 'Y')).apply(lambda x: round(x, 1))
df_Horizon_All_Features['Term_Passed'] = ((df_Horizon_All_Features.SnapshotDt - df_Horizon_All_Features.RiskCommDt)/np.timedelta64(1, 'Y')).apply(lambda x: round(x, 1))
df_Horizon_All_Features['Term_Remaining'] = ((df_Horizon_All_Features.CoverEndDt - df_Horizon_All_Features.SnapshotDt)/np.timedelta64(1, 'Y')).apply(lambda x: round(x, 1))

# Set a flag for policies with less or equal to 5 and less than or equal to 10 years term remaining.
df_Horizon_All_Features['TermRemainingLTEQ_5'] = '0'
df_Horizon_All_Features['TermRemainingLTEQ_10'] = '0'
df_Horizon_All_Features.loc[df_Horizon_All_Features['Term_Remaining'] <= 5, 'TermRemainingLTEQ_5'] = '1'
df_Horizon_All_Features.loc[df_Horizon_All_Features['Term_Remaining'] <= 10, 'TermRemainingLTEQ_10'] = '1'

# Calculate teh age at each snapshot
df_Horizon_All_Features['Age'] = ((df_Horizon_All_Features.SnapshotDt - df_Horizon_All_Features.DOB)/np.timedelta64(1, 'Y'))

# Calculate the anniversary month
df_Horizon_All_Features['AnniversaryMth'] = df_Horizon_All_Features['AnniversaryDt'].dt.month

print(df_Horizon_All_Features.dtypes)

# No displays to protect PII data
#df_Horizon_All_Features.head(5)

ContractNumber                   object
ProdCat                          object
ProdCode                         object
PolicyStatus                     object
CoverEndDt               datetime64[ns]
SnapshotDt               datetime64[ns]
Switch_Flag                       int32
PolicyFeeType                    object
PricingVariant                  float64
PayFreq                           int32
AnniversaryDt            datetime64[ns]
IndexationPercent                 int64
IndexationStartDt        datetime64[ns]
PremHolStartDt           datetime64[ns]
PremHolEndDt             datetime64[ns]
Premium                         float64
PremiumCalcMethod                object
ServicingBroker                  object
InitialBroker                    object
BrokerCategory                   object
InitialRate                     float64
RenewalRate                     float64
RiskCommDt               datetime64[ns]
VIPType                          object
Nationality                      object


## Drop all non predictive features and write to a csv file.

In [41]:
# Delete obsolete or PII columns
df_Horizon_All_Features = df_Horizon_All_Features.drop(['CoverEndDt','AnniversaryDt','PremHolStartDt','PremHolEndDt','IndexationStartDt','RiskCommDt','DOB'],axis=1)

print(df_Horizon_All_Features.dtypes)

ContractNumber                   object
ProdCat                          object
ProdCode                         object
PolicyStatus                     object
SnapshotDt               datetime64[ns]
Switch_Flag                       int32
PolicyFeeType                    object
PricingVariant                  float64
PayFreq                           int32
IndexationPercent                 int64
Premium                         float64
PremiumCalcMethod                object
ServicingBroker                  object
InitialBroker                    object
BrokerCategory                   object
InitialRate                     float64
RenewalRate                     float64
VIPType                          object
Nationality                      object
IsSmoker                         object
PoliticallyExposed               object
BirthPlace                       object
Gender                           object
MaritalStatus                    object
PensionDisability               float64


In [42]:
df_Horizon_All_Features.isna().any()

ContractNumber           False
ProdCat                  False
ProdCode                 False
PolicyStatus             False
SnapshotDt               False
Switch_Flag              False
PolicyFeeType            False
PricingVariant            True
PayFreq                  False
IndexationPercent        False
Premium                  False
PremiumCalcMethod        False
ServicingBroker           True
InitialBroker             True
BrokerCategory            True
InitialRate               True
RenewalRate               True
VIPType                   True
Nationality               True
IsSmoker                  True
PoliticallyExposed        True
BirthPlace                True
Gender                    True
MaritalStatus             True
PensionDisability         True
WOPDisability             True
AddressCity               True
PartnerType               True
EUSanctioned              True
AddressDistrict           True
Postal_Code               True
Postal_Code_Name          True
District

In [43]:
df_Horizon_All_Features.isna().any()

ContractNumber           False
ProdCat                  False
ProdCode                 False
PolicyStatus             False
SnapshotDt               False
Switch_Flag              False
PolicyFeeType            False
PricingVariant            True
PayFreq                  False
IndexationPercent        False
Premium                  False
PremiumCalcMethod        False
ServicingBroker           True
InitialBroker             True
BrokerCategory            True
InitialRate               True
RenewalRate               True
VIPType                   True
Nationality               True
IsSmoker                  True
PoliticallyExposed        True
BirthPlace                True
Gender                    True
MaritalStatus             True
PensionDisability         True
WOPDisability             True
AddressCity               True
PartnerType               True
EUSanctioned              True
AddressDistrict           True
Postal_Code               True
Postal_Code_Name          True
District

In [44]:
df_Horizon_All_Features['Postal_Code'] = df_Horizon_All_Features['Postal_Code'].fillna(0)
df_Horizon_All_Features['PricingVariant'] = df_Horizon_All_Features['PricingVariant'].fillna(0)
df_Horizon_All_Features['ServicingBroker'] = df_Horizon_All_Features['ServicingBroker'].fillna('')
df_Horizon_All_Features['InitialBroker'] = df_Horizon_All_Features['InitialBroker'].fillna('')
df_Horizon_All_Features['BrokerCategory'] = df_Horizon_All_Features['BrokerCategory'].fillna('')
df_Horizon_All_Features['InitialRate'] = df_Horizon_All_Features['InitialRate'].fillna(0)
df_Horizon_All_Features['RenewalRate'] = df_Horizon_All_Features['RenewalRate'].fillna(0)
df_Horizon_All_Features['Term'] = df_Horizon_All_Features['Term'].fillna(0)
df_Horizon_All_Features['Term_Passed'] = df_Horizon_All_Features['Term_Passed'].fillna(0)
df_Horizon_All_Features['Term_Remaining'] = df_Horizon_All_Features['Term_Remaining'].fillna(0)
df_Horizon_All_Features['PensionDisability'] = df_Horizon_All_Features['PensionDisability'].fillna(0)
df_Horizon_All_Features['WOPDisability'] = df_Horizon_All_Features['WOPDisability'].fillna(0)
df_Horizon_All_Features['VIPType'] = df_Horizon_All_Features['VIPType'].fillna('')
df_Horizon_All_Features['Nationality'] = df_Horizon_All_Features['Nationality'].fillna('')
df_Horizon_All_Features['IsSmoker'] = df_Horizon_All_Features['IsSmoker'].fillna(0)
df_Horizon_All_Features['PoliticallyExposed'] = df_Horizon_All_Features['PoliticallyExposed'].fillna('')
df_Horizon_All_Features['BirthPlace'] = df_Horizon_All_Features['BirthPlace'].fillna('')
df_Horizon_All_Features['Gender'] = df_Horizon_All_Features['Gender'].fillna('')
df_Horizon_All_Features['MaritalStatus'] = df_Horizon_All_Features['MaritalStatus'].fillna('')
df_Horizon_All_Features['AddressCity'] = df_Horizon_All_Features['AddressCity'].fillna('')
df_Horizon_All_Features['PartnerType'] = df_Horizon_All_Features['PartnerType'].fillna('')
df_Horizon_All_Features['EUSanctioned'] = df_Horizon_All_Features['EUSanctioned'].fillna('')
df_Horizon_All_Features['AddressDistrict'] = df_Horizon_All_Features['AddressDistrict'].fillna('')
df_Horizon_All_Features['Postal_Code_Name'] = df_Horizon_All_Features['Postal_Code_Name'].fillna('')
df_Horizon_All_Features['District'] = df_Horizon_All_Features['District'].fillna('')
df_Horizon_All_Features['State'] = df_Horizon_All_Features['State'].fillna('')
df_Horizon_All_Features['VN_MILIEU_CODE'] = df_Horizon_All_Features['VN_MILIEU_CODE'].fillna('')
df_Horizon_All_Features['VN_TARGET_GROUP_CODE'] = df_Horizon_All_Features['VN_TARGET_GROUP_CODE'].fillna('')
df_Horizon_All_Features['VN_MILIEU_DESCRIPTION'] = df_Horizon_All_Features['VN_MILIEU_DESCRIPTION'].fillna('')
df_Horizon_All_Features['VP_MILIEU_DESCRIPTION'] = df_Horizon_All_Features['VP_MILIEU_DESCRIPTION'].fillna('')
df_Horizon_All_Features['AnniversaryMth'] = df_Horizon_All_Features['AnniversaryMth'].fillna(0)
df_Horizon_All_Features['Age'] = df_Horizon_All_Features['Age'].fillna(0)

df_Horizon_All_Features.isna().any()

ContractNumber           False
ProdCat                  False
ProdCode                 False
PolicyStatus             False
SnapshotDt               False
Switch_Flag              False
PolicyFeeType            False
PricingVariant           False
PayFreq                  False
IndexationPercent        False
Premium                  False
PremiumCalcMethod        False
ServicingBroker          False
InitialBroker            False
BrokerCategory           False
InitialRate              False
RenewalRate              False
VIPType                  False
Nationality              False
IsSmoker                 False
PoliticallyExposed       False
BirthPlace               False
Gender                   False
MaritalStatus            False
PensionDisability        False
WOPDisability            False
AddressCity              False
PartnerType              False
EUSanctioned             False
AddressDistrict          False
Postal_Code              False
Postal_Code_Name         False
District

In [45]:
df_Horizon_All_Features.isna().any()

ContractNumber           False
ProdCat                  False
ProdCode                 False
PolicyStatus             False
SnapshotDt               False
Switch_Flag              False
PolicyFeeType            False
PricingVariant           False
PayFreq                  False
IndexationPercent        False
Premium                  False
PremiumCalcMethod        False
ServicingBroker          False
InitialBroker            False
BrokerCategory           False
InitialRate              False
RenewalRate              False
VIPType                  False
Nationality              False
IsSmoker                 False
PoliticallyExposed       False
BirthPlace               False
Gender                   False
MaritalStatus            False
PensionDisability        False
WOPDisability            False
AddressCity              False
PartnerType              False
EUSanctioned             False
AddressDistrict          False
Postal_Code              False
Postal_Code_Name         False
District

In [46]:
# Write out to a comma separated values file.
# giving directory name
filename = input_path + '\Full_Horizon_All_Features.csv'
df_Horizon_All_Features.to_csv(filename, encoding='utf-8', index=False)

print(df_Horizon_All_Features.dtypes)

# No displays to protect PII data
#df_Horizon_All_Features.head(10)

ContractNumber                   object
ProdCat                          object
ProdCode                         object
PolicyStatus                     object
SnapshotDt               datetime64[ns]
Switch_Flag                       int32
PolicyFeeType                    object
PricingVariant                  float64
PayFreq                           int32
IndexationPercent                 int64
Premium                         float64
PremiumCalcMethod                object
ServicingBroker                  object
InitialBroker                    object
BrokerCategory                   object
InitialRate                     float64
RenewalRate                     float64
VIPType                          object
Nationality                      object
IsSmoker                         object
PoliticallyExposed               object
BirthPlace                       object
Gender                           object
MaritalStatus                    object
PensionDisability               float64


# Clean up dataframes to release memory.

In [47]:
%who_ls DataFrame 

['df_Horizon_All_Features']

In [48]:
del df_Horizon_All_Features

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')


2024-08-05 21:45:19.213205
svmem(total=16756752384, available=9296678912, percent=44.5, used=7460073472, free=9296678912)
